In [2]:
import random
import torch
import math
import numpy as np

# Dataset class, from `torch.utils.data.Dataset`
class Dataset(torch.utils.data.Dataset):
    # constructor
    def __init__(self, X, Y):
        self.X = X #torch.from_numpy(X).float() #.cuda() # if gpu
        self.Y = Y #torch.from_numpy(Y).long()  #.cuda() # if gpu
    # length of the hole dataset
    def __len__(self):
        return len(self.X)
    # Get a single item
    def __getitem__(self, ix):
        return self.X[ix], self.Y[ix]


In [9]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)


epochs = 2000
batch_size = 1000
#batches = len(xx) // batch_size


# create Dataset instance
dataset = Dataset(xx,y)

# put Dataset in dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# To train we can use:
for x, y in iter(dataloader):
    print(f'the train data looks like: {x[:1]} ...')
    print(f'the y data looks like: {y[:1]} ...')
    

the train data looks like: tensor([[0.7465, 0.5573, 0.4160]]) ...
the y data looks like: tensor([0.6791]) ...
the train data looks like: tensor([[ -3.0944,   9.5756, -29.6311]]) ...
the y data looks like: tensor([-0.0471]) ...


In [15]:
# https://pytorch.org/docs/stable/nn.functional.html
import torch.nn.functional as F

class Model(torch.nn.Module):
    
    # constructor
    def __init__(self, inputs, hidden, outputs):
        
        # Call constructor of parent class
        super(Model, self).__init__()
        
        self.inputs = inputs
        self.hidden = hidden
        self.outputs = outputs
        
        # Layers: https://pytorch.org/docs/stable/nn.html
        self.fc1 = torch.nn.Linear(inputs, hidden)
        
        self.fc2 = torch.nn.Linear(hidden, outputs)
        
    # Logic for forward pass
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Model
model = Model(3, 60, 1)


# Loss functions: 
#criterion = torch.nn.MSELoss(reduction='sum')
#criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.MSELoss()

# Optimizers: https://pytorch.org/docs/stable/optim.html
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


for t in range(epochs):
    
    # Batch
    running_loss = []
    
    for x, y in iter(dataloader):

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)

        #y_pred = torch.flatten(y_pred, start_dim=0) # if I use reduction

        # Compute and print loss
        loss = criterion(y_pred, y)
        running_loss.append(loss.item())
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
    if t % 100 == 0:    # print every 10 epoch
        print(f'The loss in batch: {t} are: {np.mean(running_loss)}')
        
list(model.children())

The loss in batch: 0 are: 3.3065197467803955
The loss in batch: 100 are: 2.5539608001708984
The loss in batch: 200 are: 1.9958108067512512
The loss in batch: 300 are: 1.5795783400535583
The loss in batch: 400 are: 1.2658033967018127
The loss in batch: 500 are: 1.0309223532676697
The loss in batch: 600 are: 0.8571739792823792
The loss in batch: 700 are: 0.7334748804569244
The loss in batch: 800 are: 0.6483313739299774
The loss in batch: 900 are: 0.5922291874885559
The loss in batch: 1000 are: 0.5567356646060944
The loss in batch: 1100 are: 0.5355309247970581
The loss in batch: 1200 are: 0.5236673951148987
The loss in batch: 1300 are: 0.5173700451850891
The loss in batch: 1400 are: 0.5140111446380615
The loss in batch: 1500 are: 0.5122471451759338
The loss in batch: 1600 are: 0.5112721621990204
The loss in batch: 1700 are: 0.510453850030899
The loss in batch: 1800 are: 0.5097896158695221
The loss in batch: 1900 are: 0.5095250010490417


[Linear(in_features=3, out_features=60, bias=True),
 Linear(in_features=60, out_features=1, bias=True)]